# Demonstrate Seq2Seq Wrapper with Cornell Movie Dialog Corpus

In [9]:
import tensorflow as tf
import numpy as np

# preprocessed data
from datasets.cornell_corpus import data
import data_utils

In [10]:
import importlib
importlib.reload(data)

<module 'datasets.cornell_corpus.data' from '/userhome/student/csaky/chatbots/practical_seq2seq-master/datasets/cornell_corpus/data.py'>

In [11]:
# load data from pickle and npy files
metadata, idx_q, idx_a = data.load_data(PATH='datasets/cornell_corpus/')
(trainX, trainY), (testX, testY), (validX, validY) = data_utils.split_dataset(idx_q, idx_a)

In [12]:
# parameters 
xseq_len = trainX.shape[-1]
yseq_len = trainY.shape[-1]
batch_size = 32
xvocab_size = len(metadata['idx2w'])  
yvocab_size = xvocab_size
emb_dim = 1024

In [15]:
import seq2seq_wrapper

In [16]:
importlib.reload(seq2seq_wrapper)
model = seq2seq_wrapper.Seq2Seq(xseq_len=xseq_len,
                               yseq_len=yseq_len,
                               xvocab_size=xvocab_size,
                               yvocab_size=yvocab_size,
                               ckpt_path='ckpt/cornell_corpus/',
                               emb_dim=emb_dim,
                               num_layers=3
                               )

<log> Building Graph </log>

In [17]:
val_batch_gen = data_utils.rand_batch_gen(validX, validY, 32)
test_batch_gen = data_utils.rand_batch_gen(testX, testY, 32)
train_batch_gen = data_utils.rand_batch_gen(trainX, trainY, batch_size)

In [ ]:
sess = model.train(train_batch_gen, val_batch_gen)


<log> Training started </log>

Model saved to disk at iteration #1000
valid   loss : 1.955774
train   loss : 1.884649
lr      rate : 0.000906

Model saved to disk at iteration #2000
valid   loss : 1.975578
train   loss : 1.828991
lr      rate : 0.000821


In [65]:
sess = model.restore_last_session()

In [66]:
input_ = test_batch_gen.__next__()[0]
output = model.predict(sess, input_)
print(output.shape)

(128, 25, 8002)


In [12]:
replies = []
for ii, oi in zip(input_.T, output):
    q = data_utils.decode(sequence=ii, lookup=metadata['idx2w'], separator=' ')
    decoded = data_utils.decode(sequence=oi, lookup=metadata['idx2w'], separator=' ').split(' ')
    if decoded.count('unk') == 0:
        #if decoded not in replies:
            print('q : [{0}]; a : [{1}]'.format(q, ' '.join(decoded)))
            replies.append(decoded)

q : [frightening thought]; a : [i dont know]
q : [whys that]; a : [you know what you want]
q : [im sorry]; a : [i dont know]
q : [open that door damn you]; a : [i dont know]
q : [what do you mean]; a : [i dont know]
q : [andy before we begin id just like to say everyone in this room is your friend]; a : [what are you talking about]
q : [yeah i understand hes been up here the last six weeks]; a : [i dont know what you mean]
q : [they had me from the gate]; a : [yes sir]
q : [yes hes fine too you saw them at easter]; a : [yes sir]
q : [i need to i need to believe something this is the first time ive ever come up against anything i couldnt understand]; a : [i dont know]
q : [dont get mental man]; a : [i dont know]
q : [you havent said anything]; a : [i dont know]
q : [oh good]; a : [i dont know what you want]
q : [things change]; a : [you know what you want]
q : [yeah well miss breakfast]; a : [you know what you want]
q : [my father had farm land it was the only thing we owned of any valu

In [73]:
import tensorflow as tf
import numpy as np
import random

# preprocessed data
from datasets.cornell_corpus import data
import data_utils

# load data from pickle and npy files
metadata, idx_q, idx_a = data.load_data(PATH='datasets/cornell_corpus/')
(trainX, trainY), (testX, testY), (validX, validY) = data_utils.split_dataset(idx_q, idx_a)


input_ = test_batch_gen.__next__()[0]
#input_=np.reshape(testX[0].T,(20,1))
output = model.predict(sess, input_)
#print(output.shape)
output_max=np.argmax(output,axis=2)
print(output_max[0,:])

pred=np.zeros((128,25),dtype=np.int32)
for inst in range(128):
    for word in range(25):
        #output[0,word,output[0,word,:]<0]=0
        #output[0,word,:]+=np.abs(np.amin(output[0,word,:]))
        sum_all=np.sum(np.exp(output[inst,word,:]))
        output[inst,word,:]=np.exp(output[inst,word,:])/sum_all
    
        r=random.uniform(0,1)
        s=0
        broke=False
        for i in range(8002):
            s+=output[inst,word,i]
            if s>=r:
                pred[inst,word]=i
                broke =True
                break
        if not broke:
            pred[inst,word]=8001

output=pred
print(pred[0,:])

replies = []
for ii, oi in zip(input_.T, output):
    q = data_utils.decode(sequence=ii, lookup=metadata['idx2w'], separator=' ')
    decoded = data_utils.decode(sequence=oi, lookup=metadata['idx2w'], separator=' ').split(' ')
    if decoded.count('unk') == 0:
        #if decoded not in replies:
            print('q : [{0}]; a : [{1}]'.format(q, ' '.join(decoded)))
            replies.append(decoded)

[ 2 53  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[   3  334 5121    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0]
q : [if its in here at all its in the back two unk]; a : [i probably porno]
q : [i wish you didnt have to go so early]; a : [you doesnt you talking about looked]
q : [that depends how do the unk usually kill their victims]; a : [fifteen minutes loved]
q : [what were you doing there you never told me you were singing with an r and b group]; a : [thats isnt you talking about]
q : [i mean mr green was so nice to me he took me in and]; a : [isnt told okay]
q : [you ran without us]; a : [what national look and]
q : [no its not going through the unk busted i thought unk fixed that]; a : [its know know how]
q : [unk and draw two for unk now youre beginning to live]; a : [why get sound children act used few anyway]
q : [thats crazy ill get you some applications]; a : [very when here]
q : [i w